In [79]:
import pandas as pd
from collections import Counter
import json

In [96]:
def create_jsonl(load_file, save_file):
    # load CSV file
    df = pd.read_csv(load_file)
    print(f"File {load_file.split('/')[(len(load_file.split('/'))) - 1]} contains {len(df)} rows.")
    # put training data in the right prompt format - chat-style training is used
    training_data = []
    for i in range(len(df)):
        dialogue = df['dialogue'][i]
        note = df['note'][i]
        training_data.append({"messages": [{"role": "system", "content": "You are an expert medical professional. Given a clinical dialogue, create a clinical note outlining key dialogue aspects such as 'CHIEF COMPLAINT' (or 'CC'), 'HISTORY OF PRESENT ILLNESS' ('HPI'), 'REVIEW OF SYSTEMS', 'PHYSICAL EXAMINATION', 'VITALS REVIEWED', 'RESULTS', 'ASSESSMENT AND PLAN', 'INSTRUCTIONS', 'CURRENT MEDICATIONS', 'PAST MEDICAL HISTORY', 'EXAM', 'IMPRESSION', 'PLAN', 'ASSESSMENT', 'PAST HISTORY', 'ALLERGIES', 'SOCIAL HISTORY', 'PHYSICAL EXAM', 'PROCEDURE', 'FAMILY HISTORY', 'MEDICATIONS', 'VITALS', 'MEDICAL HISTORY', 'SURGICAL HISTORY'. You will not use all of these aspects in every dialogue, vary it from dialogue to dialogue."}, {"role": "user", "content": f"Dialogue:{dialogue}"}, {"role": "assistant", "content": f"{note}"}]})
    # save in the required file format
    if not '.jsonl' in save_file:
        save_file = save_file + '.jsonl'
    with open(f"./tuning_files/{save_file}", "w") as f:
        for item in training_data:
            f.write(json.dumps(item) + "\n")


In [97]:
# create training data
create_jsonl("./clinical_visit_note_summarization_corpus/data/aci-bench/challenge_data/train.csv", "train.jsonl")
# create validation data
create_jsonl("./clinical_visit_note_summarization_corpus/data/aci-bench/challenge_data/valid.csv", "valid.jsonl")

File train.csv contains 67 rows.
File valid.csv contains 20 rows.


# DON'T USE (ROUGHS FROM TESTING)

In [ ]:
# formulated from running through all notes to find out what headings were available
template_prompt = {"messages": [{"role": "system", "content": "You are an expert medical professional. Given a clinical dialogue, create a clinical note outlining key dialogue aspects such as 'CHIEF COMPLAINT' (or 'CC'), 'HISTORY OF PRESENT ILLNESS' ('HPI'), 'REVIEW OF SYSTEMS', 'PHYSICAL EXAMINATION', 'VITALS REVIEWED', 'RESULTS', 'ASSESSMENT AND PLAN', 'INSTRUCTIONS', 'CURRENT MEDICATIONS', 'PAST MEDICAL HISTORY', 'EXAM', 'IMPRESSION', 'PLAN', 'ASSESSMENT', 'PAST HISTORY', 'ALLERGIES', 'SOCIAL HISTORY', 'PHYSICAL EXAM', 'PROCEDURE', 'FAMILY HISTORY', 'MEDICATIONS', 'VITALS', 'MEDICAL HISTORY', 'SURGICAL HISTORY'. You will not use all of these aspects in every dialogue, vary it from dialogue to dialogue."}, {"role": "user", "content": f"Dialogue:{dialogue}"}, {"role": "assistant", "content": f"{note}"}]}

In [51]:
count = 0
headings = []
for note in train_df['note']:
    note = note.split('\n')
    note = [i for i in note if i != '']
    note = [i for i in note if i.isupper()]
    note = [i for i in note if '.' not in i]
    note = [i for i in note if '/' not in i]
    note = [i for i in note if 'MSK' not in i]
    note = [i for i in note if 'HIV' not in i]
    
    note = [i.replace(':', '') for i in note]
    headings.extend(note)

headings = list(Counter(headings).keys())
print(headings)

['CHIEF COMPLAINT', 'HISTORY OF PRESENT ILLNESS', 'REVIEW OF SYSTEMS', 'PHYSICAL EXAMINATION', 'VITALS REVIEWED', 'RESULTS', 'ASSESSMENT AND PLAN', 'INSTRUCTIONS', 'CC', 'HPI', 'CURRENT MEDICATIONS', 'PAST MEDICAL HISTORY', 'EXAM', 'IMPRESSION', 'PLAN', 'ASSESSMENT', 'PAST HISTORY', 'ALLERGIES', 'SOCIAL HISTORY', 'PHYSICAL EXAM', 'PROCEDURE', 'FAMILY HISTORY', 'MEDICATIONS', 'VITALS', 'MEDICAL HISTORY', 'SURGICAL HISTORY']


In [64]:
for note in train_df['note']:
    note = note.split('\n')
    note = [i for i in note if i != '']
    for i in range(len(note)):
        for heading in headings:
            if heading in note[i]:
                if 'ABCDE' not in note[i]:
                    try:
                        print(heading, note[i+1])
                    except:
                        print(f"\nSOMETHINGHAPPENED{note[i]}")

CHIEF COMPLAINT Annual exam.
HISTORY OF PRESENT ILLNESS Martha Collins is a 50-year-old female with a past medical history significant for congestive heart failure, depression, and hypertension who presents for her annual exam. It has been a year since I last saw the patient.
REVIEW OF SYSTEMS • Ears, Nose, Mouth and Throat: Endorses nasal congestion from allergies.
PHYSICAL EXAMINATION • Cardiovascular: Grade 3/6 systolic ejection murmur.
EXAM • Cardiovascular: Grade 3/6 systolic ejection murmur.
PHYSICAL EXAM • Cardiovascular: Grade 3/6 systolic ejection murmur.
VITALS REVIEWED • Blood Pressure: Elevated.
VITALS • Blood Pressure: Elevated.
RESULTS Echocardiogram demonstrates decreased ejection fraction of 45%. Mitral regurgitation is present.
ASSESSMENT AND PLAN Martha Collins is a 50-year-old female with a past medical history significant for congestive heart failure, depression, and hypertension who presents for her annual exam.
PLAN Martha Collins is a 50-year-old female with a pa

In [4]:
dialogue = ""
cc = ""
hpi = ""
ros = ""
pe  = ""
vr = ""
ap = ""

template_prompt = {"messages": [{"role": "system", "content": 'You are an expert medical professional. Given a clinical dialogue, provide the following fields in a JSON dict, where applicable: "chief_complaint", "history_of_present_illness", "review_of_systems", "physical_examination", "vitals_reviewed", "assessment_and_plan".'}, {"role": "user", "content": f"{dialogue}"}, {"role": "assistant", "content": f'{{"chief_complaint":{cc}, "history_of_present_illness":{hpi}, "review_of_systems":{ros}, "physical_examination":{pe}, "vitals_reviewed":{vr}, "assessment_and_plan:{ap} }}'}]}

